# Configuration

In [97]:
#Parameters
PROJECT_NAME = 'ML1010-Group-Project'
ENABLE_COLAB = True

#Root Machine Learning Directory. Projects appear underneath
GOOGLE_DRIVE_MOUNT = '/content/gdrive' 
COLAB_ROOT_DIR = GOOGLE_DRIVE_MOUNT + '/MyDrive/Colab Notebooks'
COLAB_INIT_DIR = COLAB_ROOT_DIR + '/utility_files'

LOCAL_ROOT_DIR = '/home/magni/Documents/ML_Projects'
LOCAL_INIT_DIR = LOCAL_ROOT_DIR + '/utility_files'

# Bootstrap Environment

In [98]:
#add in support for utility file directory and importing
import sys
import os

if ENABLE_COLAB:
  #Need access to drive
  from google.colab import drive
  drive.mount(GOOGLE_DRIVE_MOUNT, force_remount=True)
  
  #add in utility directory to syspath to import 
  INIT_DIR = COLAB_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = COLAB_ROOT_DIR
  
else:
  #add in utility directory to syspath to import
  INIT_DIR = LOCAL_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = LOCAL_ROOT_DIR

#Import Utility Support
from jarvis import Jarvis
jarvis = Jarvis(ROOT_DIR, PROJECT_NAME)

import mv_python_utils as mvutils

Mounted at /content/gdrive
Wha...where am I?
I am awake now.

I have set your current working directory to /content/gdrive/MyDrive/Colab Notebooks/ML1010-Group-Project
The current time is 12:29
Hello sir. Extra caffeine may help.



# Setup Runtime Environment

In [99]:

if ENABLE_COLAB:
  #!pip install scipy -q
  #!pip install scikit-learn -q
  #!pip install pycaret -q
  #!pip install matplotlib -q
  #!pip install joblib -q
  #!pip install pandasql -q
  !pip install sentence_transformers -q
  !pip install flair -q
  !pip install spacy -U -q
  !pip install spacytextblob -q
  display('Google Colab enabled')
  
else:
  display('Google Colab not enabled')

#Common imports
import json
import gzip
import pandas as pd
import numpy as np
import matplotlib
import re
import nltk
import matplotlib.pyplot as plt
import math

pd.set_option('mode.chained_assignment', None)
nltk.download('stopwords')
%matplotlib inline

'Google Colab enabled'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load Data

In [100]:
jarvis.showProjectDataFiles()
dataOrig = pd.read_pickle(jarvis.DATA_DIR + "/02_working/02_NLP_ReviewTextData.pkl")

Here are all your project data files
[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project
----->** No files **

[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project/01_original
---[   gz][ json]--> Cell_Phones_and_Accessories_5.json.gz (161.24 MB)
---[   gz][ json]--> meta_Cell_Phones_and_Accessories.json.gz (343.33 MB)

[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project/02_working
[*][  pkl]---------> 02_Cellphone.pkl (46.32 MB)
[*][  pkl]---------> 02_NLP_ReviewTextData.pkl (87.00 MB)
[*][  pkl]---------> 02_NLP_SummaryData.pkl (8.32 MB)
[*][  pkl]---------> 02_NLP_TitleData.pkl (16.71 MB)

[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project/03_train
----->** No files **

[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project/04_test
----->** No files **



In [101]:
mvutils.exploreDataframe(dataOrig)

dataframe shape: (63461, 9)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63461 entries, 0 to 63460
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   uuid                   63461 non-null  object 
 1   reviewText             63428 non-null  object 
 2   reviewText_lemma       63461 non-null  object 
 3   reviewText_nouns       63461 non-null  object 
 4   reviewText_adjectives  63461 non-null  object 
 5   reviewText_verbs       63461 non-null  object 
 6   reviewText_nav         63461 non-null  object 
 7   reviewText_no_tokens   63461 non-null  float64
 8   reviewText_lenText     63461 non-null  float64
dtypes: float64(2), object(7)
memory usage: 4.4+ MB


None


Top 5 in dataframe


,uuid,reviewText,reviewText_lemma,reviewText_nouns,reviewText_adjectives,reviewText_verbs,reviewText_nav,reviewText_no_tokens,reviewText_lenText
0,0729f856-ce3f-47b8-807f-ff9c4035db49,This phone is ugly and heavy and has a terribl...,this phone be ugly and heavy and have a terrib...,phone user interface techie call Manhattan Mot...,ugly heavy terrible crazy well well well small...,drop take give want stay,phone user interface techie call Manhattan Mot...,80.0,360.0
1,7188d7ea-555e-46c7-945a-aedad841c228,This phone is ugly and heavy and has a terribl...,this phone be ugly and heavy and have a terrib...,phone user interface techie call Manhattan Mot...,ugly heavy terrible crazy well well well small...,drop take give want stay,phone user interface techie call Manhattan Mot...,80.0,360.0
2,2e500082-1402-4f7e-8295-9db960225941,I had the Samsung V205 and then I decided to t...,-PRON- have the Samsung V205 and then -PRON- d...,Samsung V205 e715 FYI people battery problem k...,few mobile great great additional little able ...,decide try must let drain charge build do will...,Samsung V205 e715 FYI people battery problem k...,503.0,2176.0
3,362c2adf-c7ae-4338-803d-dfde5101d968,I had the Samsung V205 and then I decided to t...,-PRON- have the Samsung V205 and then -PRON- d...,Samsung V205 e715 FYI people battery problem k...,few mobile great great additional little able ...,decide try must let drain charge build do will...,Samsung V205 e715 FYI people battery problem k...,503.0,2176.0
4,f78fc0b4-2a13-4fb4-ad26-0fa90bec6e5e,This is the first Samsung phone I have had and...,this be the first Samsung phone -PRON- have ha...,Samsung phone up V300 TMobile customer V300 ca...,first terrible good external well small comfor...,have measure become feed send brake take use s...,Samsung phone up V300 TMobile customer V300 ca...,346.0,1541.0



Bottom 5 in dataframe


,uuid,reviewText,reviewText_lemma,reviewText_nouns,reviewText_adjectives,reviewText_verbs,reviewText_nav,reviewText_no_tokens,reviewText_lenText
63456,f1f8e416-77b4-475b-a086-681e19b7cced,The phone was stolen could not activate at all...,the phone be steal could not activate at all ....,phone,,steal could activate tell come steal appreciate,phone steal could activate tell come steal app...,24.0,104.0
63457,21175177-d169-4fd9-b9aa-9d12410a2602,good,good,,good,,good,1.0,4.0
63458,7367c575-be5c-49f7-9cba-578449de5c05,I purchased two of them and had so many proble...,-PRON- purchase two of -PRON- and have so many...,problem screen Samsung product,many disappointing great,purchase toss glitche,problem screen Samsung product many disappoint...,35.0,180.0
63459,9d1453d7-8e91-4157-92ac-c89c4e1d72f8,excelente,excelente,excelente,,,excelente,1.0,9.0
63460,80c53191-9935-4277-a1ab-9d4b67ee9d78,This is a great Smartphone for someone who's n...,this be a great smartphone for someone who be ...,smartphone smartphone cost Samsung thing quali...,great sure cheap low few high top low able nic...,want move construct stem leave opt leave produ...,smartphone smartphone cost Samsung thing quali...,188.0,891.0


In [102]:
dataOrig = dataOrig.dropna(subset=['reviewText'])
dataOrig.reset_index(drop=True, inplace=True)
dataOrig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63428 entries, 0 to 63427
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   uuid                   63428 non-null  object 
 1   reviewText             63428 non-null  object 
 2   reviewText_lemma       63428 non-null  object 
 3   reviewText_nouns       63428 non-null  object 
 4   reviewText_adjectives  63428 non-null  object 
 5   reviewText_verbs       63428 non-null  object 
 6   reviewText_nav         63428 non-null  object 
 7   reviewText_no_tokens   63428 non-null  float64
 8   reviewText_lenText     63428 non-null  float64
dtypes: float64(2), object(7)
memory usage: 4.4+ MB


# BERT encoding to column

In [103]:
from sentence_transformers import SentenceTransformer
model_bert = SentenceTransformer('bert-base-nli-max-tokens')

In [104]:
tDf = dataOrig[0:1000]
tDf = tDf[['uuid','reviewText']]
tDf['bert_embed'] = tDf.apply(lambda x: model_bert.encode(x['reviewText']),axis=1)

In [105]:
display(tDf)

,uuid,reviewText,bert_embed
0,0729f856-ce3f-47b8-807f-ff9c4035db49,This phone is ugly and heavy and has a terribl...,"[0.21502377, 0.968202, 0.66671306, 0.58632284,..."
1,7188d7ea-555e-46c7-945a-aedad841c228,This phone is ugly and heavy and has a terribl...,"[0.21502377, 0.968202, 0.66671306, 0.58632284,..."
2,2e500082-1402-4f7e-8295-9db960225941,I had the Samsung V205 and then I decided to t...,"[0.38180575, 1.2884152, 1.6179063, 0.1890907, ..."
3,362c2adf-c7ae-4338-803d-dfde5101d968,I had the Samsung V205 and then I decided to t...,"[0.38180575, 1.2884152, 1.6179063, 0.1890907, ..."
4,f78fc0b4-2a13-4fb4-ad26-0fa90bec6e5e,This is the first Samsung phone I have had and...,"[0.29786727, 1.205265, 0.939613, 0.22213398, 0..."
...,...,...,...
995,79370e0c-4cd2-4eef-9fca-54c14019d46f,Nice phone for tech challenged husband.,"[0.5674129, 0.5512407, 2.0276635, 0.55197453, ..."
996,d0e9ff89-f418-40d0-886d-9736d85e5f3d,I like it,"[1.202109, -0.029134436, 1.6140633, 0.46964633..."
997,6b692c03-693c-4dda-b435-ace38a64cbaf,dont last to long a couple of months.,"[0.9783736, 0.29137164, 1.5603178, 0.50250316,..."
998,a671d081-07b6-4813-9b0a-12a9ff5e7622,Excellent phone Thanks,"[1.0635258, 0.19443433, 1.1134074, 0.5910351, ..."


# Flair encoding to column (sentiment)

In [106]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')

2021-12-03 17:29:57,847 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


In [107]:
def calcFlairSentiment(doc):
  sentence = Sentence(doc)
  classifier.predict(sentence)

  value = sentence.labels[0].to_dict()['value'] 
  if value == 'POSITIVE':
      return sentence.to_dict()['labels'][0]['confidence']
  else:
      return -(sentence.to_dict()['labels'][0]['confidence'])

In [108]:
tDf = dataOrig.head(10)
tDf = tDf[['uuid', 'reviewText']]
tDf.head(5)

,uuid,reviewText
0,0729f856-ce3f-47b8-807f-ff9c4035db49,This phone is ugly and heavy and has a terribl...
1,7188d7ea-555e-46c7-945a-aedad841c228,This phone is ugly and heavy and has a terribl...
2,2e500082-1402-4f7e-8295-9db960225941,I had the Samsung V205 and then I decided to t...
3,362c2adf-c7ae-4338-803d-dfde5101d968,I had the Samsung V205 and then I decided to t...
4,f78fc0b4-2a13-4fb4-ad26-0fa90bec6e5e,This is the first Samsung phone I have had and...


In [109]:
tDf['flair_sent'] = tDf.apply(lambda x: calcFlairSentiment(x['reviewText']),axis=1)

In [110]:
tDf.head(5)

,uuid,reviewText,flair_sent
0,0729f856-ce3f-47b8-807f-ff9c4035db49,This phone is ugly and heavy and has a terribl...,-0.999998
1,7188d7ea-555e-46c7-945a-aedad841c228,This phone is ugly and heavy and has a terribl...,-0.999998
2,2e500082-1402-4f7e-8295-9db960225941,I had the Samsung V205 and then I decided to t...,0.999460
3,362c2adf-c7ae-4338-803d-dfde5101d968,I had the Samsung V205 and then I decided to t...,0.999460
4,f78fc0b4-2a13-4fb4-ad26-0fa90bec6e5e,This is the first Samsung phone I have had and...,-0.999992


# spacy TextBlob sen

In [111]:
#!pip install -U spacy
import spacy
spacy.info()

{'location': '/usr/local/lib/python3.7/dist-packages/spacy',
 'pipelines': {},
 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic',
 'python_version': '3.7.12',
 'spacy_version': '3.2.0'}

In [112]:
from spacytextblob.spacytextblob import SpacyTextBlob

In [ ]:
!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [114]:
def calcSpacySent2(df, columnName):
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('spacytextblob')
    
    totalRecords = len(df)
    for i, row in df.iterrows():
        if i % 500 == 0:
            print(str(i) + " " + str("{:.1%}".format(i/totalRecords)) + " records processed for " + str(columnName))
        if (row[columnName] and len(str(row[columnName])) < 1000000):
            doc = nlp(str(row[columnName]))
            
            df.at[i, columnName + "_TB_pol"] = doc._.polarity
            df.at[i, columnName + "_TB_subj"] = doc._.subjectivity
            

In [115]:
tDf = dataOrig.head(10)
tDf = tDf[['uuid', 'reviewText']]
tDf.head(5)

,uuid,reviewText
0,0729f856-ce3f-47b8-807f-ff9c4035db49,This phone is ugly and heavy and has a terribl...
1,7188d7ea-555e-46c7-945a-aedad841c228,This phone is ugly and heavy and has a terribl...
2,2e500082-1402-4f7e-8295-9db960225941,I had the Samsung V205 and then I decided to t...
3,362c2adf-c7ae-4338-803d-dfde5101d968,I had the Samsung V205 and then I decided to t...
4,f78fc0b4-2a13-4fb4-ad26-0fa90bec6e5e,This is the first Samsung phone I have had and...


In [116]:
calcSpacySent2(tDf, 'reviewText')

0 0.0% records processed for reviewText


In [117]:
tDf.head()

,uuid,reviewText,reviewText_TB_pol,reviewText_TB_subj
0,0729f856-ce3f-47b8-807f-ff9c4035db49,This phone is ugly and heavy and has a terribl...,-0.150000,0.660000
1,7188d7ea-555e-46c7-945a-aedad841c228,This phone is ugly and heavy and has a terribl...,-0.150000,0.660000
2,2e500082-1402-4f7e-8295-9db960225941,I had the Samsung V205 and then I decided to t...,0.202422,0.508527
3,362c2adf-c7ae-4338-803d-dfde5101d968,I had the Samsung V205 and then I decided to t...,0.202422,0.508527
4,f78fc0b4-2a13-4fb4-ad26-0fa90bec6e5e,This is the first Samsung phone I have had and...,0.088914,0.562800


# Spacy sentence splits

In [118]:
tDf = dataOrig.head(10)
tDf = tDf[['uuid', 'reviewText']]
tDf.head(5)

,uuid,reviewText
0,0729f856-ce3f-47b8-807f-ff9c4035db49,This phone is ugly and heavy and has a terribl...
1,7188d7ea-555e-46c7-945a-aedad841c228,This phone is ugly and heavy and has a terribl...
2,2e500082-1402-4f7e-8295-9db960225941,I had the Samsung V205 and then I decided to t...
3,362c2adf-c7ae-4338-803d-dfde5101d968,I had the Samsung V205 and then I decided to t...
4,f78fc0b4-2a13-4fb4-ad26-0fa90bec6e5e,This is the first Samsung phone I have had and...


In [119]:
def binSpacyPolarity(polarity, numBins):
  if polarity == -1:
    return 1
  else:
    return math.ceil(((polarity + 1) / 2) * 5)

In [123]:
def splitSpacySentences(df, columnName):
  nlp = spacy.load('en_core_web_sm')
  nlp.add_pipe('spacytextblob')

  split1=[]
  split2=[]
  split3=[]
  split4=[]
  split5=[]

  totalRecords = len(df)
  for i, row in df.iterrows():
      #progress notification
      if i % 500 == 0:
          print(str(i) + " " + str("{:.1%}".format(i/totalRecords)) + " records processed for " + str(columnName))

      #is our sentence ok to process
      if (row[columnName] and len(str(row[columnName])) < 1000000):
          doc = nlp(str(row[columnName]))
          assert doc.has_annotation("SENT_START")

      #process sentences in document
      for sent in doc.sents:
          sentDoc = nlp(str(sent.text))
          #print(sent.text + ' (pol:' + str(sentDoc._.polarity) + ', subj:' + str(sentDoc._.subjectivity) + ')')
          polBin = binSpacyPolarity(sentDoc._.polarity, 5)
          if polBin == 1:
            split1.append(sent.text)
          elif polBin == 2:
            split2.append(sent.text)
          elif polBin == 3:
            split3.append(sent.text)
          elif polBin == 4:
            split4.append(sent.text)
          elif polBin == 5:
            split5.append(sent.text)
          else:
            print("Error: spacy sentence split found sentiment out of range")

      df.at[i, columnName + "_tb_star1"] = " ".join(split1)
      df.at[i, columnName + "_tb_star2"] = " ".join(split2)
      df.at[i, columnName + "_tb_star3"] = " ".join(split3)
      df.at[i, columnName + "_tb_star4"] = " ".join(split4)
      df.at[i, columnName + "_tb_star5"] = " ".join(split5)
      

In [125]:
splitSpacySentences(tDf, 'reviewText')
mvutils.exploreDataframe(tDf)

0 0.0% records processed for reviewText
dataframe shape: (10, 7)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   uuid                 10 non-null     object
 1   reviewText           10 non-null     object
 2   reviewText_tb_star1  10 non-null     object
 3   reviewText_tb_star2  10 non-null     object
 4   reviewText_tb_star3  10 non-null     object
 5   reviewText_tb_star4  10 non-null     object
 6   reviewText_tb_star5  10 non-null     object
dtypes: object(7)
memory usage: 688.0+ bytes


None


Top 5 in dataframe


,uuid,reviewText,reviewText_tb_star1,reviewText_tb_star2,reviewText_tb_star3,reviewText_tb_star4,reviewText_tb_star5
0,0729f856-ce3f-47b8-807f-ff9c4035db49,This phone is ugly and heavy and has a terribl...,This phone is ugly and heavy and has a terribl...,,Get the Moto V600 if you want T-Mobile.,"The Moto V600 is a much better phone, takes b...",
1,7188d7ea-555e-46c7-945a-aedad841c228,This phone is ugly and heavy and has a terribl...,This phone is ugly and heavy and has a terribl...,,Get the Moto V600 if you want T-Mobile. Get ...,"The Moto V600 is a much better phone, takes b...",
2,2e500082-1402-4f7e-8295-9db960225941,I had the Samsung V205 and then I decided to t...,This phone is ugly and heavy and has a terribl...,"Otherwise, it has the usual features.",Get the Moto V600 if you want T-Mobile. Get ...,"The Moto V600 is a much better phone, takes b...",and I stay with them because: 1) They are now ...
3,362c2adf-c7ae-4338-803d-dfde5101d968,I had the Samsung V205 and then I decided to t...,This phone is ugly and heavy and has a terribl...,"Otherwise, it has the usual features. Otherwis...",Get the Moto V600 if you want T-Mobile. Get ...,"The Moto V600 is a much better phone, takes b...",and I stay with them because: 1) They are now ...
4,f78fc0b4-2a13-4fb4-ad26-0fa90bec6e5e,This is the first Samsung phone I have had and...,This phone is ugly and heavy and has a terribl...,"Otherwise, it has the usual features. Otherwis...",Get the Moto V600 if you want T-Mobile. Get ...,"The Moto V600 is a much better phone, takes b...",and I stay with them because: 1) They are now ...



Bottom 5 in dataframe


,uuid,reviewText,reviewText_tb_star1,reviewText_tb_star2,reviewText_tb_star3,reviewText_tb_star4,reviewText_tb_star5
5,8c6912f7-67bc-451d-a15d-239f6c5b5ae1,This is the first Samsung phone I have had and...,This phone is ugly and heavy and has a terribl...,"Otherwise, it has the usual features. Otherwis...",Get the Moto V600 if you want T-Mobile. Get ...,"The Moto V600 is a much better phone, takes b...",and I stay with them because: 1) They are now ...
6,38229a0b-64e9-4aa6-a425-217268034027,"..and I'm from Europe, where the phones and ne...",This phone is ugly and heavy and has a terribl...,"Otherwise, it has the usual features. Otherwis...",Get the Moto V600 if you want T-Mobile. Get ...,"The Moto V600 is a much better phone, takes b...",and I stay with them because: 1) They are now ...
7,8d416a08-2709-4f95-9f79-1cab29fbb9d4,"..and I'm from Europe, where the phones and ne...",This phone is ugly and heavy and has a terribl...,"Otherwise, it has the usual features. Otherwis...",Get the Moto V600 if you want T-Mobile. Get ...,"The Moto V600 is a much better phone, takes b...",and I stay with them because: 1) They are now ...
8,ed3ab051-d151-4c19-9b02-3b52b3059af4,This phone is amazingly small and light for a ...,This phone is ugly and heavy and has a terribl...,"Otherwise, it has the usual features. Otherwis...",Get the Moto V600 if you want T-Mobile. Get ...,"The Moto V600 is a much better phone, takes b...",and I stay with them because: 1) They are now ...
9,0795c8eb-6e58-4f2d-b6b4-3673a17650f6,This phone is amazingly small and light for a ...,This phone is ugly and heavy and has a terribl...,"Otherwise, it has the usual features. Otherwis...",Get the Moto V600 if you want T-Mobile. Get ...,"The Moto V600 is a much better phone, takes b...",and I stay with them because: 1) They are now ...


In [ ]:
tDf.info()

In [122]:
print ("polarity: " + str(binSpacyPolarity(-1.0, 5)))
print ("polarity: " + str(binSpacyPolarity(-0.9, 5)))
print ("polarity: " + str(binSpacyPolarity(-0.8, 5)))
print ("polarity: " + str(binSpacyPolarity(-0.7, 5)))
print ("polarity: " + str(binSpacyPolarity(-0.6, 5)))
print ("polarity: " + str(binSpacyPolarity(-0.5, 5)))
print ("polarity: " + str(binSpacyPolarity(-0.4, 5)))
print ("polarity: " + str(binSpacyPolarity(-0.3, 5)))
print ("polarity: " + str(binSpacyPolarity(-0.2, 5)))
print ("polarity: " + str(binSpacyPolarity(-0.1, 5)))
print ("polarity: " + str(binSpacyPolarity(-0.0, 5)))
print ("polarity: " + str(binSpacyPolarity(0.1, 5)))
print ("polarity: " + str(binSpacyPolarity(0.2, 5)))
print ("polarity: " + str(binSpacyPolarity(0.3, 5)))
print ("polarity: " + str(binSpacyPolarity(0.4, 5)))
print ("polarity: " + str(binSpacyPolarity(0.5, 5)))
print ("polarity: " + str(binSpacyPolarity(0.6, 5)))
print ("polarity: " + str(binSpacyPolarity(0.7, 5)))
print ("polarity: " + str(binSpacyPolarity(0.8, 5)))
print ("polarity: " + str(binSpacyPolarity(0.9, 5)))
print ("polarity: " + str(binSpacyPolarity(1.0, 5)))

polarity: 1
polarity: 1
polarity: 1
polarity: 1
polarity: 1
polarity: 2
polarity: 2
polarity: 2
polarity: 2
polarity: 3
polarity: 3
polarity: 3
polarity: 3
polarity: 4
polarity: 4
polarity: 4
polarity: 4
polarity: 5
polarity: 5
polarity: 5
polarity: 5


In [139]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

text = 'Along with and WiFi connectivity the platform is said to offer multiday battery life'
      
doc = nlp(text)
print(len(doc))
print(len(str(doc)))


14
84
